In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px # ploty

from collections import Counter

import gc, sys
gc.enable() # 자동 가비지 수거 활성화

In [ ]:
base_path = '/content/drive/MyDrive/이어드림스쿨/Project/Final_project/2021 농산물 예측대회 AI 경진대회/2021 농산물 가격예측 AI 경진대회/프로젝트 data/'

In [ ]:
df =pd.read_parquet(base_path + 'test_modeling_data1.parquet')
df.head()

,SALEDATE,PUM_NM,KIND_NM,SAN_NM,mean_price,TOT_AMT,TOT_QTY,year,month,day,...,온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,2016-01-01,사과,후지,충북 충주,1875.0,150000.0,80.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
1,2016-01-01,사과,후지,충북 충주,1800.0,54000.0,30.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
2,2016-01-01,사과,후지,충북 충주,2200.0,99000.0,45.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
3,2016-01-01,사과,후지,충북 충주,2600.0,39000.0,15.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
4,2016-01-01,사과,후지,충북 충주,400.0,12000.0,30.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391


## 이상치 제거 후 EDA 진행

- 평균단가 : 농넷 및 뉴스 기사 확인 후 이상치 제거

- 대파 : 5000원 이하로
- 배추 : 20000원 이하로
- 무 : 3000원 이하로
- 양파 : 3000원 이하로
- 사과 : 6000원 이하로
- 마늘 : 20000원 이하로
- 건고추 : 40000원 이하로

In [ ]:
print('대파', df[(df.PUM_NM == '대파')].shape[0], df[(df.PUM_NM == '대파') & (df.mean_price > 5000)].shape[0], '비율', round(df[(df.PUM_NM == '대파') & (df.mean_price > 5000)].shape[0] / df[(df.PUM_NM == '대파')].shape[0] * 100, 4), '%') 
print('배추', df[(df.PUM_NM == '배추')].shape[0], df[(df.PUM_NM == '배추') & (df.mean_price > 20000)].shape[0], '비율', round(df[(df.PUM_NM == '배추') & (df.mean_price > 20000)].shape[0] / df[(df.PUM_NM == '배추')].shape[0] * 100, 4), '%')
print('무', df[(df.PUM_NM == '무')].shape[0], df[(df.PUM_NM == '무') & (df.mean_price > 3000)].shape[0], '비율', round(df[(df.PUM_NM == '무') & (df.mean_price > 3000)].shape[0] / df[(df.PUM_NM == '무')].shape[0] * 100,4), '%')
print('양파', df[(df.PUM_NM == '양파')].shape[0], df[(df.PUM_NM == '양파') & (df.mean_price > 3000)].shape[0], '비율', round(df[(df.PUM_NM == '양파') & (df.mean_price > 3000)].shape[0] / df[(df.PUM_NM == '양파')].shape[0] * 100, 4), '%')
print('사과', df[(df.PUM_NM == '사과')].shape[0], df[(df.PUM_NM == '사과') & (df.mean_price > 6000)].shape[0], '비율', round(df[(df.PUM_NM == '사과') & (df.mean_price > 6000)].shape[0] / df[(df.PUM_NM == '사과')].shape[0] * 100, 4), '%')
print('마늘', df[(df.PUM_NM == '마늘')].shape[0], df[(df.PUM_NM == '마늘') & (df.mean_price > 20000)].shape[0], '비율', round(df[(df.PUM_NM == '마늘') & (df.mean_price > 20000)].shape[0] / df[(df.PUM_NM == '마늘')].shape[0] * 100, 4), '%')
print('건고추', df[(df.PUM_NM == '건고추')].shape[0], df[(df.PUM_NM == '건고추') & (df.mean_price > 40000)].shape[0], '비율', round(df[(df.PUM_NM == '건고추') & (df.mean_price > 40000)].shape[0] / df[(df.PUM_NM == '건고추')].shape[0] * 100, 4), '%')

대파 2040543 13244 비율 0.649 %
배추 1921360 8269 비율 0.4304 %
무 1338449 19923 비율 1.4885 %
양파 1753780 4211 비율 0.2401 %
사과 9117455 146070 비율 1.6021 %
마늘 1466622 2204 비율 0.1503 %
건고추 39953 136 비율 0.3404 %


In [ ]:
df2 = df.copy()

In [ ]:
df2 = df2.drop(df2[(df2.PUM_NM == '대파') & (df2.mean_price > 5000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '배추') & (df2.mean_price > 20000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '무') & (df2.mean_price > 3000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '양파') & (df2.mean_price > 3000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '사과') & (df2.mean_price > 6000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '마늘') & (df2.mean_price > 20000)].index)
df2 = df2.drop(df2[(df2.PUM_NM == '건고추') & (df2.mean_price > 40000)].index)

df2 = df2.reset_index(drop=True)
df2.head()

,SALEDATE,PUM_NM,KIND_NM,SAN_NM,mean_price,TOT_AMT,TOT_QTY,year,month,day,...,온도(최고),지면온도(최저),지면온도(평균),지면온도(최고),풍속(m/s)(평균),습도(%)(평균),강수량(mm)(합계),적설량(cm)(합계),일조량(합계),일조량(평균)
0,2016-01-01,사과,후지,충북 충주,1875.0,150000.0,80.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
1,2016-01-01,사과,후지,충북 충주,1800.0,54000.0,30.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
2,2016-01-01,사과,후지,충북 충주,2200.0,99000.0,45.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
3,2016-01-01,사과,후지,충북 충주,2600.0,39000.0,15.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391
4,2016-01-01,사과,후지,충북 충주,400.0,12000.0,30.0,2016,1,1,...,5.8,-8.4,-2.404348,7.4,1.043478,68.086957,0.0,0.0,7.3,0.317391


In [ ]:
print(df.shape, df2.shape)

(17678162, 29) (17484105, 29)


## 주산지

In [ ]:
pum_lst = df2.PUM_NM.unique()
print(len(pum_lst), pum_lst)

7 ['사과' '양파' '대파' '무' '배추' '마늘' '건고추']


### 품목별 총거래량 top 30


In [ ]:
# top 30개만 뽑아봄

for i in pum_lst :
    a = df2[df2.PUM_NM == i].groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending = False)[:30]
    fig = px.pie(a, values = a.values, names = a.index)
    fig.update_layout(title_text=f'{i} 산지별 총거래량', title_x=0.5) # title을 center에 지정하는 코드
    fig.update_yaxes(tickformat=',') # y값 단위 k,m가 아닌 ,으로 숫자 그대로 반환하는 코드
    fig.show()

### 품목별 거래건수 top 30

In [ ]:
# top 30개만 뽑아봄

for i in pum_lst : 
    b = df2[df2.PUM_NM == i ]['SAN_NM'].value_counts()[:30]
    fig = px.pie(b, names = b.index, values = b.values)
    fig.update_layout(title_text = f'{i} 산지별 거래횟수', title_x=0.5) # title을 center에 지정하는 코드
    fig.update_yaxes(tickformat=',') # y값 단위 k,m가 아닌 ,으로 숫자 그대로 반환하는 코드
    fig.show()

### 품종의 산지별 평균 금액

In [ ]:
for i in pum_lst :
    c = df[df.PUM_NM == i].groupby('SAN_NM')['mean_price'].mean().sort_values(ascending = False)[:20]
    fig = px.bar(c, x=c.index, y=c.values)
    fig.update_layout(title_text=f'{i}의 산지별 평균단가 top20', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

In [ ]:
for i in pum_lst :
    c = df2[df2.PUM_NM == i].groupby('SAN_NM')['mean_price'].mean().sort_values(ascending = False)[:20]
    fig = px.bar(c, x=c.index, y=c.values)
    fig.update_layout(title_text=f'{i}의 산지별 평균단가 top20', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

## 도매시장

In [ ]:
# 해당 데이터에 도매시장 column이 없어서 전의 data로 진행함

do_df = pd.read_parquet(base_path + 'new_selcet_df.parquet_1') 
do_df['PUM_NM'] = do_df['PUM_NM'].astype('object')
do_df.columns

Index(['SALEDATE', 'WHSAL_NM', 'CMP_NM', 'PUM_NM', 'KIND_NM', 'DAN_NM',
       'POJ_NM', 'SIZE_NM', 'LV_NM', 'SAN_NM', 'DANQ', 'QTY', 'COST',
       'TOT_QTY', 'TOT_AMT', 'mean_price'],
      dtype='object')

### 품목별 도매시장 top 10

In [ ]:
pum_lst = do_df.PUM_NM.unique()
pum_lst

array(['대파', '배추', '무', '양파', '사과', '마늘', '건고추'], dtype=object)

In [ ]:
for i in pum_lst : 
    a = do_df[do_df.PUM_NM == i]['WHSAL_NM'].value_counts()[:10]
    fig = px.bar(a, x=a.index, y=a.values)
    fig.update_layout(title_text=f'{i} 도매시장 거래건수 top10', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

### 품목의 품종별 도매시장

In [ ]:
do_df['KIND_NM'] = do_df['KIND_NM'].astype('object')
do_df['KIND_NM'] = do_df['KIND_NM'].astype('category')

In [ ]:
for i in pum_lst : 
    a = do_df[do_df.PUM_NM == i]['WHSAL_NM'].value_counts()[:10]
    fig = px.bar(a, x=a.index, y=a.values)
    fig.update_layout(title_text=f'{i} 도매시장 top10', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

In [ ]:
pum_lst = do_df.PUM_NM.unique()
pum_lst

In [ ]:
do_df[do_df.PUM_NM == '사과']['KIND_NM'].value_counts()

후지        3151874
홍로        1831485
미안마       1226777
미시마        628328
기타사과       523776
           ...   
여름무             0
달랑무             0
양파(일반)          0
단무지무            0
양파(수입)          0
Name: KIND_NM, Length: 154, dtype: int64

In [ ]:
for i in pum_lst : 
    kind_lst = do_df[do_df.PUM_NM == i]['KIND_NM'].unique()
    print(len(kind_lst), kind_lst)

6 ['대파(일반)', '깐대파', '대파', '중파', '대파(수입)', '기타대파']
Categories (154, object): ['OBIR', '가을무', '간(같은)마늘', '갈라', ..., '화건', '화홍', '후지', '히로사끼']
16 ['쌈배추', '기타배추', '우거지', '절임배추', '봄배추', ..., '생채용 배추', '고냉지배추', '우거지(수입)', '배양채', '배추(수입)']
Length: 16
Categories (154, object): ['OBIR', '가을무', '간(같은)마늘', '갈라', ..., '화건', '화홍', '후지', '히로사끼']
20 ['기타무', '건무(수입)', '무말랭이', '저장무', '세척무', ..., '가을무', '무말랭이(수입)', '자색무', '갯무', '무(수입)']
Length: 20
Categories (154, object): ['OBIR', '가을무', '간(같은)마늘', '갈라', ..., '화건', '화홍', '후지', '히로사끼']
13 ['양파(일반)', '양파(수입)', '만생양파', '깐양파', '기타양파', ..., '중생양파', '조생양파', '중만생양파', '샬롯', '자주양파(수입)']
Length: 13
Categories (154, object): ['OBIR', '가을무', '간(같은)마늘', '갈라', ..., '화건', '화홍', '후지', '히로사끼']
74 ['미시마', '기꾸8', '후지', '미안마', '로얄부사', ..., '데리셔스', '노변', '명월', '메구미', '아리수']
Length: 74
Categories (154, object): ['OBIR', '가을무', '간(같은)마늘', '갈라', ..., '화건', '화홍', '후지', '히로사끼']
17 ['기타마늘', '풋마늘', '마늘쫑(수입)', '깐마늘', '마늘쫑', ..., '육쪽마늘', '저장형 난지', '햇마늘 난지', '쪽마늘', '간(같은)마늘']
Length

In [ ]:
for i in df_lst : 
    a = i.WHSAL_NM.value_counts()
    min = str(i.SALEDATE.min()).split(' ')[0]
    max = str(i.SALEDATE.max()).split(' ')[0]
    fig = px.line(a, x = a.index, y = a.values, title = f'도매시장별 거래량 ( {min} ~ {max} )')
    fig.update_yaxes(tickformat=',')
    fig.show()

### 품목 품종별 평균 금액

In [ ]:
df['KIND_NM'] = df['KIND_NM'].astype('object')
df['KIND_NM'] = df['KIND_NM'].astype('category')

In [ ]:
# 원본 데이터

for i in pum_lst :
    c = df[df.PUM_NM == i].groupby('KIND_NM')['mean_price'].mean().sort_values(ascending = False)[:20]    
    fig = px.bar(c, x=c.index, y=c.values)
    fig.update_layout(title_text=f'{i}의 품종별 평균단가 top20', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

In [ ]:
# 이상치 대략적으로 제거한 데이터

for i in pum_lst :
    c = df2[df2.PUM_NM == i].groupby('KIND_NM')['mean_price'].mean().sort_values(ascending = False)[:20]    
    fig = px.bar(c, x=c.index, y=c.values)
    fig.update_layout(title_text=f'{i}의 품종별 평균단가 top20', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

## 거래량

### 품종별 거래량

In [ ]:
a = df.groupby('PUM_NM')['TOT_QTY'].sum()
fig = px.pie(a, names=a.index, values=a.values)
fig.update_layout(title_text='품종별 거래량', title_x=0.5)
fig.update_yaxes(tickformat=',')
fig.show()

In [ ]:
for i in pum_lst :
    c = df[df.PUM_NM == i].groupby('KIND_NM')['TOT_QTY'].sum().sort_values(ascending = False)[:20]    
    fig = px.bar(c, x=c.index, y=c.values)
    fig.update_layout(title_text=f'{i}의 품종별 거래량 top20', title_x=0.5)
    fig.update_yaxes(tickformat=',')
    fig.show()

### 수입 품종 거래량

In [ ]:
df